In [1]:
import sys
import os
import json
import pickle
import torch

In [2]:
from config import SECRET_KEY, SYSTEM_PROMPT, FUNCTION_STRUCTURE
import openai
openai.api_key = SECRET_KEY

In [4]:
from GPT_to_env import answer_to_training_data
from env_to_GPT import get_human_feedback

In [5]:
gpt_model = 'gpt-4o'
temperature = 1 # Degree of randomness of GPT's output

In [17]:
directory = 'results/italy'
exp_name = 'experiment_name_step0'

In [18]:
with open(os.path.join(directory, f'{exp_name}_trajectory.pkl'), 'rb') as f:
    trajectory = pickle.load(f)
human_feedback = get_human_feedback(directory, exp_name)
print(human_feedback)

['The goal is to replicate the Italian flag on the front face of the cube. From state 4 until 10, it is all wrong.']


In [19]:
# Create the user prompt
trajectory_json = json.dumps(trajectory)
user_prompt = f'feedback: "{human_feedback[0]}". \n {trajectory_json}'

In [20]:
gptResponse = openai.chat.completions.create(
                            model = gpt_model,
                            temperature= temperature,
                            messages= [{'role': 'system', 'content': SYSTEM_PROMPT},
                                       {"role": "user", "content": user_prompt}],
                                functions=[FUNCTION_STRUCTURE],
                                    function_call= { "name": "getReward" })

In [21]:
json_data = gptResponse.choices[0].message.function_call.arguments
answer_data = json.loads(json_data)
[x_train, y_train] = answer_to_training_data(answer_data)
# Save the tensors
torch.save(x_train, os.path.join(directory, f'{exp_name}_x_train.pt'))
torch.save(y_train, os.path.join(directory, f'{exp_name}_y_train.pt'))

In [22]:
answer_data

{'states': [{'state': [['G', 'G', 'G'], ['W', 'W', 'W'], ['R', 'R', 'R']],
   'reward': 1},
  {'state': [['Y', 'R', 'W'], ['Y', 'B', 'W'], ['G', 'R', 'G']], 'reward': -1},
  {'state': [['Y', 'R', 'W'], ['B', 'O', 'B'], ['G', 'R', 'G']], 'reward': -1},
  {'state': [['W', 'B', 'G'], ['R', 'O', 'R'], ['Y', 'B', 'G']], 'reward': -1},
  {'state': [['W', 'W', 'G'], ['R', 'Y', 'R'], ['Y', 'B', 'G']], 'reward': -1},
  {'state': [['W', 'W', 'G'], ['R', 'Y', 'R'], ['O', 'R', 'W']], 'reward': -1},
  {'state': [['W', 'W', 'G'], ['R', 'Y', 'R'], ['Y', 'B', 'G']], 'reward': -1},
  {'state': [['W', 'W', 'G'], ['R', 'Y', 'R'], ['Y', 'B', 'G']],
   'reward': -1}]}